We will attempt to use the dataset from Kaggle and data clean it here for us to be able to finally test the ASA-GNN framework. Thanks to the notebook provided by https://www.kaggle.com/code/tuttifrutti/isolating-a-cardid

In [69]:
import pandas as pd
import numpy as np

In [70]:
train_transaction = pd.read_csv('/Users/tontbhencha-arporn/Downloads/ASA-GNN Fraud Datasets/train_transaction.csv')
train_identity = pd.read_csv('/Users/tontbhencha-arporn/Downloads/ASA-GNN Fraud Datasets/train_identity.csv')
train_transaction = train_transaction.merge(train_identity, how='left', left_on='TransactionID',right_on='TransactionID')
del train_identity

In [71]:
train_transaction.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M


In [72]:
import itertools
import math
import networkx as nx

#function to create keys based on multiple columns
def create_key(df, cols, name_new_col):
    '''
    df: pandas dataframe
    cols: list of columns composing the key
    name_new_col: name given to the new column
    '''
    df.loc[:,name_new_col] = ''
    for col in cols:
        df.loc[:,name_new_col] = df.loc[:,name_new_col] + df.loc[:,col].astype(str)
    return df  

def truncate(f, n):
    return math.floor(f * 10 ** n) / 10 ** n  

def merge(list1, list2): 
    merged_list = [[p1, p2] for idx1, p1 in enumerate(list1)  
    for idx2, p2 in enumerate(list2) if idx1 == idx2] 
    return merged_list   



In [73]:
train_transaction['day'] = train_transaction['TransactionDT']/(3600*24)
train_transaction['D1minusday'] = (train_transaction['D1']-train_transaction['day']).replace(np.nan, -9999).map(int)
colsID = ['card1','card2','card3','card4','card5','card6','D1minusday','ProductCD']
train_transaction = create_key(train_transaction, colsID, 'cardID_D1')

In [74]:
train_transaction['cardID_D1'].value_counts()

cardID_D1
15775481.0150.0mastercard102.0credit-129S         1414
9500321.0150.0visa226.0debit84W                    480
7919194.0150.0mastercard166.0debit-92W             439
7919194.0150.0mastercard166.0debit-124W            282
7919194.0150.0mastercard202.0debit-34W             242
                                                  ... 
16452514.0150.0mastercard224.0credit-57H             1
16426399.0150.0american express118.0credit-57R       1
3602119.0150.0visa195.0credit-57R                    1
4029594.0150.0visa226.0debit-57W                     1
12037595.0150.0mastercard224.0debit-182W             1
Name: count, Length: 171059, dtype: int64

In [75]:
train_transaction['V307'] = train_transaction['V307'].fillna(0)
train_transaction['V307plus'] = train_transaction['V307']+train_transaction['TransactionAmt']

In [76]:
train_transaction['V307rtrunc'] = train_transaction['V307'].apply(lambda x: truncate(x,3))
train_transaction['V307round'] = train_transaction['V307'].apply(lambda x: round(x,3))
train_transaction['V307plusround'] = train_transaction['V307plus'].apply(lambda x: round(x,4))
train_transaction['V307plusroundtrunc'] = train_transaction['V307plusround'].apply(lambda x: truncate(x,3))
train_transaction['V307plusround'] = train_transaction['V307plus'].apply(lambda x: round(x,3))
train_transaction['V307trunc2'] = train_transaction['V307'].apply(lambda x: truncate(x,2))
train_transaction['V307plustrunc2'] = train_transaction['V307plus'].apply(lambda x: truncate(x,2))
train_transaction['TransactionAmttrunq'] = train_transaction['TransactionAmt'].apply(lambda x: round(x,3))

In [77]:
def find_groups(aa):
    group_list = []
    
    #get the couples by existstrun
    list1 = aa['V307plusroundtrunc'].tolist()
    list2 = aa['V307rtrunc'].tolist()
    kv = []
    res = [[list(filter(lambda z: list1[z]==x, range(len(list1)))),list(filter(lambda z: list2[z]==x, range(len(list2))))] for x in list1 if x in list2] #find the pairs
    res= [list(map(kv.append,map(list,(itertools.product(*sublist))))) for sublist in res] #drop duplicates from list of list
    res = list(map(list, set(map(lambda i: tuple(i), kv)))) #create list of couple indexes
    list1 = aa.iloc[[i[0] for i in res]]['TransactionID'].tolist()
    list2 = aa.iloc[[i[1] for i in res]]['TransactionID'].tolist()
    liste_existstrun = merge(list1, list2)


    #get the couples by existsroundtrunc
    list1 = aa['V307plusroundtrunc'].tolist()
    list2 = aa['V307round'].tolist()
    kv = []
    res = [[list(filter(lambda z: list1[z]==x, range(len(list1)))),list(filter(lambda z: list2[z]==x, range(len(list2))))] for x in list1 if x in list2] #find the pairs
    res= [list(map(kv.append,map(list,(itertools.product(*sublist))))) for sublist in res] #drop duplicates from list of list
    res = list(map(list, set(map(lambda i: tuple(i), kv)))) #create list of couple indexes
    list1 = aa.iloc[[i[0] for i in res]]['TransactionID'].tolist()
    list2 = aa.iloc[[i[1] for i in res]]['TransactionID'].tolist()
    liste_existsroundtrunc = merge(list1, list2)

    #get the couples by existsroundtrunc
    list1 = aa['V307plusround'].tolist()
    list2 = aa['V307round'].tolist()
    kv = []
    res = [[list(filter(lambda z: list1[z]==x, range(len(list1)))),list(filter(lambda z: list2[z]==x, range(len(list2))))] for x in list1 if x in list2] #find the pairs
    res= [list(map(kv.append,map(list,(itertools.product(*sublist))))) for sublist in res] #drop duplicates from list of list
    res = list(map(list, set(map(lambda i: tuple(i), kv)))) #create list of couple indexes
    list1 = aa.iloc[[i[0] for i in res]]['TransactionID'].tolist()
    list2 = aa.iloc[[i[1] for i in res]]['TransactionID'].tolist()
    liste_existsroundround = merge(list1, list2)


    #get the couples by existsroundtrunc
    list1 = aa['V307trunc2'].tolist()
    list2 = aa['V307plustrunc2'].tolist()
    kv = []
    res = [[list(filter(lambda z: list1[z]==x, range(len(list1)))),list(filter(lambda z: list2[z]==x, range(len(list2))))] for x in list1 if x in list2] #find the pairs
    res= [list(map(kv.append,map(list,(itertools.product(*sublist))))) for sublist in res] #drop duplicates from list of list
    res = list(map(list, set(map(lambda i: tuple(i), kv)))) #create list of couple indexes
    list1 = aa.iloc[[i[0] for i in res]]['TransactionID'].tolist()
    list2 = aa.iloc[[i[1] for i in res]]['TransactionID'].tolist()
    liste_existstrunc2 = merge(list1, list2)


    #get the couples by existsamount
    list1 = aa['TransactionAmttrunq'].tolist()
    list2 = aa['V307round'].tolist()
    kv = []
    res = [[list(filter(lambda z: list1[z]==x, range(len(list1)))),list(filter(lambda z: list2[z]==x, range(len(list2))))] for x in list1 if x in list2] #find the pairs
    res= [list(map(kv.append,map(list,(itertools.product(*sublist))))) for sublist in res] #drop duplicates from list of list
    res = list(map(list, set(map(lambda i: tuple(i), kv)))) #create list of couple indexes
    list1 = aa.iloc[[i[0] for i in res]]['TransactionID'].tolist()
    list2 = aa.iloc[[i[1] for i in res]]['TransactionID'].tolist()
    liste_existsamount = merge(list1, list2)

    #get by exact same amount
    a=[]
    liste_sameamount = aa.groupby('TransactionAmt')['TransactionID'].apply(list).tolist()
    res = [list(map(a.append, map(list,zip(i, i[1:] + i[:1])))) for i in liste_sameamount]

    group_list.extend(liste_existstrun)
    group_list.extend(liste_existsroundtrunc)
    group_list.extend(liste_existsamount)
    group_list.extend(liste_existsroundround)
    group_list.extend(liste_existstrunc2)

    group_list.extend(a)

    L = group_list
    G = nx.Graph()
    G.add_nodes_from(sum(L, []))
    q = [[(s[i],s[i+1]) for i in range(len(s)-1)] for s in L]
    for i in q:
        G.add_edges_from(i)
    group_list = [list(i) for i in nx.connected_components(G)]
    return group_list

In [78]:
card_group = train_transaction[train_transaction.cardID_D1=='16136204.0185.0visa138.0debit108C']
groups_found = find_groups(card_group)
groups_found

[[3030465, 3009027, 3026025, 3008981, 3008631, 3025980],
 [3271808, 3271837, 3271815],
 [3173544, 3173773, 3173777, 3173458, 3173468],
 [3437224, 3537388, 3518333, 3437190],
 [3445490, 3445471]]

In [79]:
def create_card_ids(train_transaction, find_groups):
    """
    Creates a unique card_ID for each transaction based on cardID_D1 groups
    and V307 patterns identified by find_groups().
    """
    # Initialize the card_ID column with None
    train_transaction['card_ID'] = None
    
    # Counter for unique card IDs
    card_id_counter = 0
    
    # Get all unique ccardID_D1 values
    unique_card_id_d1 = train_transaction['cardID_D1'].unique()
    
    # Process each cardID_D1 group
    for card_d1 in unique_card_id_d1:
        # Filter dataframe for this specific cardID_D1
        mask = train_transaction['cardID_D1'] == card_d1
        df_subset = train_transaction[mask]
        
        # Get the groups (list of lists of indices)
        groups = find_groups(df_subset)
        
        # Assign a unique card_ID to each group
        for group_transaction_ids in groups:

            transaction_mask = train_transaction['TransactionID'].isin(group_transaction_ids)
            train_transaction.loc[transaction_mask, 'card_ID'] = card_id_counter
            card_id_counter += 1
    
    return train_transaction


In [80]:
train_transaction = create_card_ids(train_transaction, find_groups)

We have now gotten a column to unqiuely identify a card called "card_ID". Now, we need to create a way to uniquely identify a user called "user_ID".

In [81]:
colsID = ['id_19','id_20','id_31','DeviceInfo']
train_transaction = create_key(train_transaction, colsID, 'user_ID_mock')

In [82]:
#check the number of unique users first
train_transaction['user_ID_mock'].value_counts()

user_ID_mock
nannannannan                                                    449851
633.0533.0edge 16.0Windows                                        1102
216.0214.0ie 11.0 for desktoprv:11.0                              1044
266.0507.0chrome 63.0Windows                                       758
266.0325.0chrome 63.0Windows                                       486
                                                                 ...  
574.0507.0chrome 64.0 for androidSM-G900M Build/LRX21T               1
548.0521.0chrome 63.0 for androidVenue                               1
256.0324.0chrome 63.0Windows                                         1
391.0600.0chrome 63.0 for androidSM-G935P Build/NRD90M               1
417.0595.0chrome 66.0 for androidRNE-L03 Build/HUAWEIRNE-L03         1
Name: count, Length: 33650, dtype: int64

In [103]:
counts = train_transaction['user_ID_mock'].dropna().value_counts()

# users that appear between 10 and 25 times (inclusive)
users_10_25 = counts[(counts >= 10) & (counts <= 25)].index

# filtered dataframe containing only those users
filtered = train_transaction[train_transaction['user_ID_mock'].isin(users_10_25)]

filtered['user_ID_mock'].value_counts()

user_ID_mock
100.0533.0chrome 66.0Windows                 25
529.0299.0mobile safari genericiOS Device    25
633.0391.0mobile safari genericiOS Device    25
100.0549.0chrome 62.0MacOS                   25
100.0533.0chrome 65.0Windows                 25
                                             ..
202.0107.0chrome 64.0Windows                 10
410.0278.0chrome 63.0Windows                 10
193.0333.0edge 16.0Windows                   10
542.0277.0chrome 63.0Windows                 10
410.0142.0chrome 62.0Windows                 10
Name: count, Length: 1640, dtype: int64

In [84]:
train_transaction[train_transaction['user_ID_mock']=='633.0533.0edge 16.0Windows']

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V307plus,V307rtrunc,V307round,V307plusround,V307plusroundtrunc,V307trunc2,V307plustrunc2,TransactionAmttrunq,card_ID,user_ID_mock
106634,3093634,0,2128047,100.0,R,2616,327.0,150.0,discover,102.0,...,100.0,0.0,0.0,100.0,100.0,0.0,100.0,100.0,89424,633.0533.0edge 16.0Windows
216910,3203910,0,5069345,200.0,H,7918,555.0,150.0,visa,226.0,...,200.0,0.0,0.0,200.0,200.0,0.0,200.0,200.0,148656,633.0533.0edge 16.0Windows
368553,3355553,0,9156842,100.0,R,7309,399.0,150.0,american express,118.0,...,100.0,0.0,0.0,100.0,100.0,0.0,100.0,100.0,219237,633.0533.0edge 16.0Windows
368556,3355556,0,9156897,100.0,R,7309,399.0,150.0,american express,118.0,...,200.0,100.0,100.0,200.0,200.0,100.0,200.0,100.0,219237,633.0533.0edge 16.0Windows
368558,3355558,0,9156948,100.0,R,7309,399.0,150.0,american express,118.0,...,300.0,200.0,200.0,300.0,300.0,200.0,300.0,100.0,219237,633.0533.0edge 16.0Windows
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574505,3561505,0,15271575,30.0,S,15775,481.0,150.0,mastercard,102.0,...,145795.0,145765.0,145765.0,145795.0,145795.0,145765.0,145795.0,30.0,249289,633.0533.0edge 16.0Windows
577530,3564530,0,15364252,120.0,S,15775,481.0,150.0,mastercard,102.0,...,145730.0,145610.0,145610.0,145730.0,145730.0,145610.0,145730.0,120.0,249291,633.0533.0edge 16.0Windows
579667,3566667,0,15451381,85.0,S,15775,481.0,150.0,mastercard,102.0,...,145695.0,145610.0,145610.0,145695.0,145695.0,145610.0,145695.0,85.0,249290,633.0533.0edge 16.0Windows
581436,3568436,0,15527523,30.0,S,15775,481.0,150.0,mastercard,102.0,...,145670.0,145640.0,145640.0,145670.0,145670.0,145640.0,145670.0,30.0,249289,633.0533.0edge 16.0Windows


In [85]:
train_transaction['V264'] = train_transaction['V264'].fillna(0)
train_transaction['V264plus'] = train_transaction['V264']+train_transaction['TransactionAmt']

In [94]:
train_transaction['V264'].value_counts()

V264
0.000000      550444
100.000000      2462
50.000000       1660
200.000000      1613
150.000000      1363
               ...  
179.623398         1
18.764099          1
82.961403          1
365.391693         1
499.955414         1
Name: count, Length: 13358, dtype: int64

In [95]:
train_transaction['V264plus'].value_counts()

V264plus
59.000000     30582
117.000000    28935
107.950000    23954
57.950000     23600
100.000000    17939
              ...  
640.460302        1
169.193404        1
148.995399        1
99.330199         1
400.780000        1
Name: count, Length: 38283, dtype: int64

In [96]:
train_transaction['V264rtrunc'] = train_transaction['V264'].apply(lambda x: truncate(x,3))
train_transaction['V264round'] = train_transaction['V264'].apply(lambda x: round(x,3))
train_transaction['V264plusround'] = train_transaction['V264plus'].apply(lambda x: round(x,4))
train_transaction['V264plusroundtrunc'] = train_transaction['V264plusround'].apply(lambda x: truncate(x,3))
train_transaction['V264plusround'] = train_transaction['V264plus'].apply(lambda x: round(x,3))
train_transaction['V264trunc2'] = train_transaction['V264'].apply(lambda x: truncate(x,2))
train_transaction['V264plustrunc2'] = train_transaction['V264plus'].apply(lambda x: truncate(x,2))
train_transaction['TransactionAmttrunq'] = train_transaction['TransactionAmt'].apply(lambda x: round(x,3))

In [88]:
def find_user_groups(aa):
    group_list = []
    
    #get the couples by existstrun
    list1 = aa['V264plusroundtrunc'].tolist()
    list2 = aa['V264rtrunc'].tolist()
    kv = []
    res = [[list(filter(lambda z: list1[z]==x, range(len(list1)))),list(filter(lambda z: list2[z]==x, range(len(list2))))] for x in list1 if x in list2] #find the pairs
    res= [list(map(kv.append,map(list,(itertools.product(*sublist))))) for sublist in res] #drop duplicates from list of list
    res = list(map(list, set(map(lambda i: tuple(i), kv)))) #create list of couple indexes
    list1 = aa.iloc[[i[0] for i in res]]['TransactionID'].tolist()
    list2 = aa.iloc[[i[1] for i in res]]['TransactionID'].tolist()
    liste_existstrun = merge(list1, list2)


    #get the couples by existsroundtrunc
    list1 = aa['V264plusroundtrunc'].tolist()
    list2 = aa['V264round'].tolist()
    kv = []
    res = [[list(filter(lambda z: list1[z]==x, range(len(list1)))),list(filter(lambda z: list2[z]==x, range(len(list2))))] for x in list1 if x in list2] #find the pairs
    res= [list(map(kv.append,map(list,(itertools.product(*sublist))))) for sublist in res] #drop duplicates from list of list
    res = list(map(list, set(map(lambda i: tuple(i), kv)))) #create list of couple indexes
    list1 = aa.iloc[[i[0] for i in res]]['TransactionID'].tolist()
    list2 = aa.iloc[[i[1] for i in res]]['TransactionID'].tolist()
    liste_existsroundtrunc = merge(list1, list2)

    #get the couples by existsroundtrunc
    list1 = aa['V264plusround'].tolist()
    list2 = aa['V264round'].tolist()
    kv = []
    res = [[list(filter(lambda z: list1[z]==x, range(len(list1)))),list(filter(lambda z: list2[z]==x, range(len(list2))))] for x in list1 if x in list2] #find the pairs
    res= [list(map(kv.append,map(list,(itertools.product(*sublist))))) for sublist in res] #drop duplicates from list of list
    res = list(map(list, set(map(lambda i: tuple(i), kv)))) #create list of couple indexes
    list1 = aa.iloc[[i[0] for i in res]]['TransactionID'].tolist()
    list2 = aa.iloc[[i[1] for i in res]]['TransactionID'].tolist()
    liste_existsroundround = merge(list1, list2)


    #get the couples by existsroundtrunc
    list1 = aa['V264trunc2'].tolist()
    list2 = aa['V264plustrunc2'].tolist()
    kv = []
    res = [[list(filter(lambda z: list1[z]==x, range(len(list1)))),list(filter(lambda z: list2[z]==x, range(len(list2))))] for x in list1 if x in list2] #find the pairs
    res= [list(map(kv.append,map(list,(itertools.product(*sublist))))) for sublist in res] #drop duplicates from list of list
    res = list(map(list, set(map(lambda i: tuple(i), kv)))) #create list of couple indexes
    list1 = aa.iloc[[i[0] for i in res]]['TransactionID'].tolist()
    list2 = aa.iloc[[i[1] for i in res]]['TransactionID'].tolist()
    liste_existstrunc2 = merge(list1, list2)


    #get the couples by existsamount
    list1 = aa['TransactionAmttrunq'].tolist()
    list2 = aa['V264round'].tolist()
    kv = []
    res = [[list(filter(lambda z: list1[z]==x, range(len(list1)))),list(filter(lambda z: list2[z]==x, range(len(list2))))] for x in list1 if x in list2] #find the pairs
    res= [list(map(kv.append,map(list,(itertools.product(*sublist))))) for sublist in res] #drop duplicates from list of list
    res = list(map(list, set(map(lambda i: tuple(i), kv)))) #create list of couple indexes
    list1 = aa.iloc[[i[0] for i in res]]['TransactionID'].tolist()
    list2 = aa.iloc[[i[1] for i in res]]['TransactionID'].tolist()
    liste_existsamount = merge(list1, list2)

    #get by exact same amount
    a=[]
    liste_sameamount = aa.groupby('TransactionAmt')['TransactionID'].apply(list).tolist()
    res = [list(map(a.append, map(list,zip(i, i[1:] + i[:1])))) for i in liste_sameamount]

    group_list.extend(liste_existstrun)
    group_list.extend(liste_existsroundtrunc)
    group_list.extend(liste_existsamount)
    group_list.extend(liste_existsroundround)
    group_list.extend(liste_existstrunc2)

    group_list.extend(a)

    L = group_list
    G = nx.Graph()
    G.add_nodes_from(sum(L, []))
    q = [[(s[i],s[i+1]) for i in range(len(s)-1)] for s in L]
    for i in q:
        G.add_edges_from(i)
    group_list = [list(i) for i in nx.connected_components(G)]
    return group_list

In [106]:
def create_user_ids(train_transaction, find_groups):
    """
    Creates a unique user_ID for each transaction based on user_ID_mock groups
    and V264 patterns identified by find_groups().
    """
    # Initialize the user_ID column with None
    train_transaction['user_ID'] = None
    # Counter for unique user IDs
    user_id_counter = 0
    # Get all unique user_ID_mock values (excluding NaN)
    unique_user_id_mock = train_transaction['user_ID_mock'].dropna().unique()
    
    # Process each user_ID_mock group
    for user_mock in unique_user_id_mock:
        # Filter dataframe for this specific user_ID_mock
        mask = train_transaction['user_ID_mock'] == user_mock
        df_subset = train_transaction[mask].copy()
        
        # Get the groups (list of lists of TransactionIDs)
        groups = find_groups(df_subset)
        
        # Assign a unique user_ID to each group
        for group_transaction_ids in groups:
            transaction_mask = train_transaction['TransactionID'].isin(group_transaction_ids)
            train_transaction.loc[transaction_mask, 'user_ID'] = user_id_counter
            user_id_counter += 1
    
    return train_transaction

In [112]:
filtered_train = train_transaction[train_transaction["user_ID_mock"]=='nannannannan']
filtered_train.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,card_ID,user_ID_mock,V264plus,V264rtrunc,V264round,V264plusround,V264plusroundtrunc,V264trunc2,V264plustrunc2,user_ID
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,0,nannannannan,68.5,0.0,0.0,68.5,68.5,0.0,68.5,None
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,1,nannannannan,29.0,0.0,0.0,29.0,29.0,0.0,29.0,None
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,2,nannannannan,59.0,0.0,0.0,59.0,59.0,0.0,59.0,None
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,10,nannannannan,50.0,0.0,0.0,50.0,50.0,0.0,50.0,None
5,2987005,0,86510,49.0,W,5937,555.0,150.0,visa,226.0,...,15,nannannannan,49.0,0.0,0.0,49.0,49.0,0.0,49.0,None


In [110]:
filtered_train.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,card_ID,user_ID_mock,V264plus,V264rtrunc,V264round,V264plusround,V264plusroundtrunc,V264trunc2,V264plustrunc2,user_ID
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,0,nannannannan,68.5,0.0,0.0,68.5,68.5,0.0,68.5,26
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,1,nannannannan,29.0,0.0,0.0,29.0,29.0,0.0,29.0,7
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,2,nannannannan,59.0,0.0,0.0,59.0,59.0,0.0,59.0,24
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,10,nannannannan,50.0,0.0,0.0,50.0,50.0,0.0,50.0,21
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,14,542.0144.0samsung browser 6.2SAMSUNG SM-G892A ...,50.0,0.0,0.0,50.0,50.0,0.0,50.0,61


In [111]:
user_group = train_transaction[train_transaction["user_ID_mock"]=='nannannannan']
groups_found = find_user_groups(user_group)
groups_found

KeyboardInterrupt: 